---
title: Data Exploration
format:
    html:
        embed-resources: true
---

# Exploratory Data Analysis

In order to do machine learning with our data, we need to develop a thorough understanding of the data and the relationships between the variables. We will do Exploratory Data Analysis (EDA) to understand the data and to prepare for modeling.

## Text Data

To begin, we can explore the textual data gathered that relates to psychosis and cannabis. This text data will be essential for answering questions regarding public sentiment related to the impact of cannabis on psychosis and/or schizophrenia.

### Reddit



In [6]:
library(tidyverse, quietly = TRUE, warn.conflicts = FALSE)
wiki_data <- read_delim("../data/clean_data/wiki_clean_data.txt", delim = "\n", col_names = FALSE)

In [ ]:
wiki_data